<a href="https://colab.research.google.com/github/ItzWarLockOp/Google-Colab-Scriptz/blob/main/Copy%20Folder%20to%20MyDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<b>COPY FOLDER TO MY DRIVE 
<Center> <img src="https://edtechteacher.org/wp-content/uploads/brizy/imgs/Google-Drive-Folder-414x336x1x0x412x336x1680337051.png" height="60" >

___________________________________________

In [ ]:

#@title <b><Center> Mount Google Drive </Center>
#@markdown <Center> <img src="https://kstatic.googleusercontent.com/files/e20dbc1db7b6eab22fb2d07df8043f8a74b301dbb8c176e6a1992b2dc229c5ae94b5c260cf549c41712fa40b7e639166c6a8461303a27c2a3b9c55b8ca155ec3" class="icon" height="40" alt="Icon">

from google.colab import drive
drive.mount('/content/drive')

___________________________________________

In [ ]:
#@markdown <Center> <img src="https://www.google.com/images/branding/googlelogo/2x/googlelogo_light_color_160x56dp.png" class="icon" height="50" alt="Icon" > 
#@title <Center> <b> Authenticate Your Google Account 

!pip install -q tqdm
!pip install -q google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2

from google.colab import auth
from googleapiclient.discovery import build

# Authenticate and create the Drive API client
auth.authenticate_user()
drive_service = build('drive', 'v3')

print("User Authenticated Successfully!")

___________________________________________

#<b>📲Enter Shared Folder Path & Destination Path📁

In [ ]:

from tqdm import tqdm

def copy_shared_folder_to_target_folder(shared_folder_path, destination_path):
    # Get the shared folder ID from the path
    query = f"name='{shared_folder_path.split('/')[-1]}' and mimeType='application/vnd.google-apps.folder'"
    shared_folder = drive_service.files().list(q=query, fields='files(id)').execute()
    shared_folder_id = shared_folder['files'][0]['id']
    shared_folder_name = shared_folder_path.split('/')[-1]

    # Get the destination folder ID from the path
    query = f"name='{destination_path.split('/')[-1]}' and mimeType='application/vnd.google-apps.folder'"
    destination_folder = drive_service.files().list(q=query, fields='files(id)').execute()
    destination_folder_id = destination_folder['files'][0]['id']

    # Create a folder with the same name in the destination folder
    folder_metadata = {
        'name': shared_folder_name,
        'parents': [destination_folder_id],
        'mimeType': 'application/vnd.google-apps.folder'
    }
    new_folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
    new_folder_id = new_folder.get('id')

    # Copy the contents of the shared folder to the new folder
    query = f"'{shared_folder_id}' in parents"
    shared_folder_contents = drive_service.files().list(q=query, fields='files(id)').execute()
    num_files = len(shared_folder_contents.get('files', []))

    with tqdm(total=num_files, desc='Copying files') as pbar:
        for file in shared_folder_contents.get('files', []):
            file_id = file.get('id')
            drive_service.files().copy(fileId=file_id, body={'parents': [new_folder_id]}).execute()
            pbar.update(1)

    print('\nShared folder copied successfully!🎉')

shared_folder_path = '/content/drive/MyDrive/Shared Folder' #@param {type:"string"}
destination_path = '/content/drive/MyDrive/Destination' #@param {type:"string"}

copy_shared_folder_to_target_folder(shared_folder_path, destination_path)